In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install koreanize_matplotlib

In [ ]:
import koreanize_matplotlib

%config InlineBackend.figure_format = 'retina'

In [ ]:
df = pd.read_excel('Online Retail.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'Online Retail.xlsx'

In [ ]:
for i in df.select_dtypes(include=['object']).columns:
  df[i] = df[i].astype(str)

In [ ]:
# parquet_file_path = '/content/drive/MyDrive/online_retail.parquet'
# df.to_parquet(parquet_file_path,index=False)

In [ ]:
df = pd.read_parquet(parquet_file_path)
df.head()
df.tail()

In [ ]:
df.info()

In [ ]:
# category 형태가 속도가 더 빨라서 변환
for i in df.select_dtypes(include=['object','datetime64[ns]']).columns :
  df[i] = df[i].astype('category')

Attribute Information :
- invoiceNo : 송장번호, 해당 거래에 할당된 6자리 숫자
- StockCode : 제품 코드, 각 고유 제품에 할당된 5자리 숫자
- Description : 제품 이름
- Quantity : 거래당 각 제품의 수량
- InvoiceDate : 송장 날짜 및 시간, 숫자, 각 거래가 생성된 날짜 및 시간
- UnitPrice : 단가, 숫자, 스털링(영국 화폐) 단위의 제품 가격
- CustomerID : 고객 번호, 해당 고객에게 고유하게 할당된 5자리 정수
- Country : 국가 이름, 해당 고객이 거주하는 고객의 이름

In [ ]:
df.describe()

In [ ]:
# shift + tab
# df.describe?
df.describe(exclude='number')

In [ ]:
df.isnull().sum()
# 비회원 구매자가 많은 것 같다.

In [ ]:
df.isnull().mean()*100

In [ ]:
 plt.figure(figsize=(12,4))
 sns.heatmap(df.isnull(), cmap='gray')

In [ ]:
df.hist(figsize=(10,6), bins=100);
# ; 로그 안나옴
# 값이 몰려있다. 이상치 확인 후 처리 필요

In [ ]:
# 주문금액 컬럼을 만들자
df['Totalprice'] = df['Quantity'] * df['UnitPrice']

In [ ]:
df['CustomerID'].isnull().value_counts(normalize=True)
df['CustomerID'].isnull().value_counts(1)

In [ ]:
df.loc[~df['CustomerID'].notnull(),'Country'].value_counts()
# true = ~true
# ~pd.Series([True,False])

In [ ]:
df['isRegistered'] = df['CustomerID'].notnull()

In [ ]:
# pd.crosstab = 교차표
pd.crosstab(df['Country'],df['isRegistered']).sort_values(True,ascending=False)

*구매 취소 비율 확인*

- invoiceno 컬럼의 값에 숫자가 아닌 'c'가 함께 있는 값

- df['iscancel'] 컬럼 만들기


In [ ]:
df['InvoiceCode'] = df['InvoiceNo'].str.replace('[0-9]','',regex=True)

In [ ]:
df['InvoiceCode'].value_counts()
df['isCancel'] = (df['InvoiceCode'] == 'C')

In [ ]:
# 회원 , 비회원 구매건중 취소건 비교
# C는 취손데 A는 뭘까
pd.crosstab(df['isRegistered'],df['InvoiceCode'])

In [ ]:
df[df['InvoiceCode'] == 'A']

In [ ]:
# 고객별 구매취소 비율
# group by와 테이블 pivot하는 방식
# 밑 두개는 같은 결과를 준다
pd.crosstab(df['CustomerID'],df['isCancel'])
df.groupby(['CustomerID','isCancel'])['InvoiceNo'].count().unstack()

In [ ]:
customer_cancel = pd.crosstab(df['CustomerID'],df['isCancel'])
customer_cancel.columns = ['정상','취소']
customer_cancel['취소비율'] = customer_cancel['취소']/ customer_cancel['정상']

In [ ]:
# inf는 0으로 나눴을때 나오는 값
customer_cancel.nlargest(10,'취소비율')
customer_cancel.nlargest(10,'취소')

In [ ]:
df[df['InvoiceCode'] == 'C'].hist();

In [ ]:
df[df['InvoiceCode'] == 'C'].describe()

In [ ]:
# 특정 고객에 기록 보기
df[df['CustomerID'] == 14911]

In [ ]:
# 제품 별 구매 취소 비율
stock_cancel = pd.crosstab(df['StockCode'],df['isCancel'])
df_stock_desc = df[['StockCode','Description']].drop_duplicates('StockCode').dropna().set_index('StockCode')

In [ ]:
# merge, join
# stock_cancel.merge(df_stock_desc,left_index=True, right_index=True)
stock_cancel = stock_cancel.join(df_stock_desc)

In [ ]:
stock_cancel.columns = ['정상','취소','설명']

In [ ]:
stock_cancel.nlargest(10,'정상')

In [ ]:
# 고객 id가 없는 주문과 취소 주문은 전체 데이터에서 제외

In [ ]:
df.shape
df['CustomerID'].notnull()
df.loc[df['Quantity'] < 0,'InvoiceCode'].value_counts()
# 취소값 말고 다른 1336 확인

In [ ]:
df.loc[(df['Quantity'] < 0) & (df['InvoiceCode'] != 'C'),'Description'].value_counts()

In [ ]:
df.loc[(df['Quantity'] < 0) & (df['InvoiceCode'] != 'C'),'Quantity'].describe()

In [ ]:
df[df['Description'] == 'MEMO BOARD RETROSPOT  DESIGN']

In [ ]:
df_valid = df.loc[df['CustomerID'].notnull() &
            (df['Quantity'] > 0 ) &
            (df['UnitPrice'] > 0 ) ]

In [ ]:
df.shape, df_valid.shape

In [ ]:
df_valid = df_valid.drop_duplicates().copy()

In [ ]:
df_valid['InvoiceDate'] = pd.to_datetime(df_valid['InvoiceDate'])

In [ ]:
df_valid['InvoiceYear'] = df_valid['InvoiceDate'].dt.year
df_valid['InvoiceMonth'] = df_valid['InvoiceDate'].dt.month
df_valid['InvoiceDay'] = df_valid['InvoiceDate'].dt.day
df_valid['InvoiceDow'] = df_valid['InvoiceDate'].dt.dayofweek

In [ ]:
df_valid['InvoiceYM'] = df_valid['InvoiceDate'].astype(str).str[:7]

In [ ]:
df_valid['InvoiceTime'] = df_valid['InvoiceDate'].dt.time
df_valid['InvoiceHour'] = df_valid['InvoiceDate'].dt.hour

In [ ]:
df_valid.hist(figsize=(15,8),bins=70);

In [ ]:
# countplot
sns.countplot(data=df_valid, x='InvoiceYear',hue='InvoiceYear')

In [ ]:
sns.countplot(data=df_valid,x='InvoiceMonth',hue='InvoiceMonth')

In [ ]:
sns.countplot(data=df_valid, x='InvoiceMonth', hue='InvoiceYear')

In [ ]:
# 요일별 빈도수
sns.countplot(data=df_valid, x='InvoiceDow')
# 5(토욜)이 없다.

In [ ]:
day_name = [w for w in '월화수목금토일']
day_name

In [ ]:
day_name.remove('토')

In [ ]:
dow_count = df_valid['InvoiceDow'].value_counts().sort_index()
dow_count.index = day_name
dow_count.plot.bar(figsize=(12,3), rot=0)

In [ ]:
df_valid['InvoiceDayName'] = df_valid['InvoiceDow'].map(lambda x :'월화수목금토일'[x])

In [ ]:
sns.countplot(data=df_valid, x='InvoiceDayName',order=list('월화수목금토일'),hue='InvoiceDayName')
# 목요일이 가장 주문이 많다.

In [ ]:
# 시간대별
plt.figure(figsize=(12,4))
sns.countplot(data=df_valid, x='InvoiceHour',hue='InvoiceHour')

In [ ]:
sns.pointplot(data=df_valid, x='InvoiceHour', y='Totalprice', estimator=len, errorbar=None).set_title('시간대별 주문량')

In [ ]:
# 매출액은 10시 12시 에 가장 높다.
sns.pointplot(data=df_valid, x='InvoiceHour', y='Totalprice', estimator=sum, errorbar=None).set_title('시간대별 매출액')

In [ ]:
list('월화수목금일')

In [ ]:
# 시간별,요일별 구매 빈도수
hour_dow = pd.crosstab(df_valid['InvoiceHour'],df_valid['InvoiceDayName'])
hour_dow = hour_dow[list('월화수목금일')]
hour_dow

In [ ]:
# 일요일 12시 가장높고 대체로 12시에 구매 빈도수가 높다.
hour_dow.style.background_gradient(cmap='brg',axis=None).format('{:,}')
# print(plt.colormaps()) -- 색깔 고르기

In [ ]:
# annot=숫자표기, fmt=표기방법
sns.heatmap(hour_dow,cmap='Purples',annot=True,fmt=',.0f')

In [ ]:
# 시간,요일별 구매 주문
# subplot
hour_dow.plot()
plt.figure(figsize=(12,4))
hour_dow.plot(subplots=True,kind='area');

**고객**
- ARPU
  - 가입한 서비스에 대해 가입자 1명이 특정 기간 동안 지출한 평균 금액
  -ARPU = 매출/중복을 제외한 순수 활동 사용자 수
- ARPPU
  - 지불 유저 1명 당 한 달에 결제하는 평균 금액을 산정한 수치


In [ ]:
# ARPPU - CUSTOMERID를 사용 할때 NUNIQUE 사용
# ARPPU
arppu = df_valid.groupby(['InvoiceYM']).agg({'Totalprice':'sum',
                                             'CustomerID':'nunique'})
arppu['ARPPU'] = arppu['Totalprice'] / arppu['CustomerID']
arppu

In [ ]:
arppu['Totalprice'].plot(figsize=(10,3),title='매출액');

In [ ]:
arppu['ARPPU'].plot(figsize=(10,3),title='ARPPU');

In [ ]:
arppu['CustomerID'].plot(figsize=(10,3), title='월별 고객 수 MAU')

In [ ]:
# df_valid(유효고객, 유효주문)내 고객별 (customerID ) 구매 (InvoiceNo) 빈도수 계산
# 고객별 구매 빈도수, 평균 구매 금액, 총 구매금액
# cust_agg
cust_agg = df_valid.groupby('CustomerID').agg({'InvoiceNo' : 'count',
                                               'Totalprice' : ['mean','sum']})
cust_agg

In [ ]:
cust_agg.describe()

In [ ]:
cust_agg.hist(bins=50);

***mau***


In [ ]:
mau = df_valid.groupby('InvoiceYM')['CustomerID'].nunique()
mau.plot.bar(title='MAU')

In [ ]:
# 월별 주문건 ,중복을 제외한 주문제품 종류 수, 고객 수, 총 주문금액
df_valid.groupby('InvoiceYM').agg({'InvoiceNo':'count',
                                   'StockCode':'nunique',
                                   'CustomerID' : 'nunique',
                                   'UnitPrice' : 'mean',
                                   'Quantity' : 'sum',
                                   'Totalprice' : 'sum'})

***리텐션을 구하기 위한 월단위 데이터 전처리***


In [ ]:
# 최근 구매일 - 최초 구매일로 첫 구매 후 몇달 후 구매인지를 알 수 있다.
# 최초 구매일(Invoicedatefirst)를 구해보자
df_valid['Invoicedatefirst'] = df_valid.groupby(['CustomerID'])['InvoiceDate'].transform('min')

In [ ]:
df_valid[['CustomerID','InvoiceDate','Invoicedatefirst']].sample(5)

***첫 구매일로 부터 몇 달째 구매인가?***


In [ ]:
# 연도별 차이 year_diff, 월별 차이 month_diff 를 구한다.
year_diff = df_valid['InvoiceDate'].dt.year - df_valid['Invoicedatefirst'].dt.year
month_diff = df_valid['InvoiceDate'].dt.month - df_valid['Invoicedatefirst'].dt.month
df_valid['cohort'] = (year_diff * 12) + month_diff +1
df_valid['cohort']

In [ ]:
# 서서히 낮아지기 보단 첫 구매후 급격하게 떨어진다.
sns.countplot(data=df_valid, x='cohort',hue='cohort')

**잔존 빈도 구하기**

In [ ]:
df_valid['Invoicedatefirstym'] = df_valid['Invoicedatefirst'].astype(str).str[:7]

In [ ]:
# invoicedatefirstym, cohort 로 그룹화 후 customerid의 유일값에 대한 빈도수를 구한다.
cohort_count = df_valid.groupby(['Invoicedatefirstym','cohort'])['CustomerID'].nunique().unstack()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(cohort_count,cmap='Greens',annot=True,fmt='.0f')

In [ ]:
cohort_count[1].plot(rot=0,title='월별 신규 고객 수')

*잔존율 구하기*

In [ ]:
# div, axis=0을 사용해 첫 달을 기준으로 나머지 달을 나눌 수 있다.
cohort_norm = cohort_count.div(cohort_count[1],axis=0)

In [ ]:
sns.heatmap(cohort_norm,cmap='Oranges',annot=True)

월별 매출액 리텐션

In [ ]:
cohort_totalprice = df_valid.groupby(['Invoicedatefirstym','cohort'])['Totalprice'].sum().unstack()

In [ ]:
# 2010-12월에 첫구매한 고객들이 꾸준히 높은 매출을 유지하고 있으며
# 갈수록 매출액은 떨어지고 있다
plt.figure(figsize=(22,8))
sns.heatmap(cohort_totalprice,cmap='Blues',annot=True,fmt=',.0f')

**RFM 분석**
- RECENCY , FREQUENCY, MONETARY
- 거래의 최근성, 거래빈도, 거래규모 기준을 가지고 고객을 입체적으로 분류, 파악하는 분석
- SCORING 기법 : RFM 요인을 각각 5등급으로 동간격으로 분류하는 방법
- 파레토 법칙 : 상위 20% 고객이 80%에 매출을 차지 한다.


In [ ]:
raw_valid = df[(df['CustomerID'].notnull()) &
               (df['Quantity'] > 0) &
               (df['UnitPrice'] > 0)].copy()

In [ ]:
raw_valid.shape

In [ ]:
raw_valid['TotalPrice'] = raw_valid['Quantity'] * raw_valid['UnitPrice']

In [ ]:
raw_valid.describe()

In [ ]:
raw_valid[raw_valid['TotalPrice'] > 160000]

In [ ]:
raw_valid = raw_valid[raw_valid['TotalPrice'] < 160000].copy()

In [ ]:
# keep = 'first' 중복 데이터 발견시 처음 데이터 사용 (first, last, False)로 설정 가능
raw_valid.duplicated(keep='first')
# 중복된 데이터
raw_valid[raw_valid.duplicated(keep=False)].sort_values(['InvoiceNo','StockCode'])

In [ ]:
# 중복된 데이터 제거
df2 = raw_valid.drop_duplicates().copy()

In [ ]:
df2['InvoiceDate'] = pd.to_datetime(df2['InvoiceDate'])

In [ ]:
# Recency 계산을 위해 해당 주문에서 가장 최근 구매가 일어난 시간을 가져옵니다.
# 최근 거래 기준일(last_timestamp)을 만들기 위해 timedelta로 날짜를 더해줍니다 (최소값:1 설정)
last_timestamp = df2['InvoiceDate'].max() + dt.timedelta(days=1)

고객별 recency, frequency, monetary 값 구하기

In [ ]:
# recency : 최근 거래 기준일 (last_timestamp)과 고객별 최근 구매한 날짜 x.max()의 차이
# frequency : 구매 빈도수
# monetary : 총 구매 금액
rfm = df2.groupby('CustomerID').agg({'InvoiceDate' : lambda x: (last_timestamp - x.max()).days,
                              'InvoiceNo': 'count',
                              'TotalPrice': 'sum'})

In [ ]:
rfm = rfm.rename(columns={'InvoiceDate':'Recency', 'InvoiceNo':'Frequency','TotalPrice':'Monetary'})

In [ ]:
print(rfm.describe())
rfm.hist(bins=60);

In [ ]:
# plt.figure(figsize=(6,3))
# sns.boxplot(rfm)
# 스케일 차이가 크다
'''sns.boxplot(data=rfm,x='Recency')
sns.boxplot(data=rfm,x='Frequency')
sns.boxplot(data=rfm,x='Monetary')'''

RFM 모형
- SCORING 기법 : RFM 요인을 5가지로 분류하는 방법
- 현재 개발된 RFM 모형은 크게 4가지로 분류
  - 모델1 : RFM 각 요소의 20% RULE 적용
  - 모델2 : 비율 척도에 의한 양적인 정도의 차이에 따른 등간격의 5등급 분류
  - 모델3 : 상하 20%를 제외한 등간격 척도에 의한 그룹 분류
  - 모델4 : 군집 분석에 의한 각 요소 5개의 그룹 분류

In [ ]:
# pandas에 cut, qcut 알아보기 (hist를 그릴 때 설정하는 bin과 같은 느낌)
# cut은 주로 연속적인 숫자 데이터를 일정한 구간으로 나누어 범주형 변수로 변환 할 때 사용
# qcut 구간에 동일한 수의 데이터가 포함되도록 한다. 따라서 데이터 분포에 따라 구간 경계가 자동으로 조정 된다.
# 차이점 : cut은 사용자가 지정한 구간 크기나 경계에 따라 나뉘고, qcut은 동일한 개수의 데이터가 포함되도록 나눈다.
# 쉽게 말하면 시험이 절대평가면 cut으로 등급을 나누고 상대평가면 qcut으로 등급을 나눈다!

In [ ]:
# Recency는 최근일수록 높은 스코러을 갖도록 한다.
# Frequency, Monetary는 값이 클 수록 높은 스코어를 갖도록 한다.
r_labels = list(range(5,0,-1))
f_labels = list(range(1,6))
m_labels = list(range(1,6,))

In [ ]:
rfm
cut_size = 5

In [ ]:
r_qcut= pd.qcut(x=rfm['Recency'],q=cut_size,labels=r_labels)
f_qcut = pd.qcut(x=rfm['Frequency'],q=cut_size,labels=f_labels)
m_qcut = pd.qcut(x=rfm['Monetary'],q=cut_size,labels=m_labels)

In [ ]:
f_qcut.value_counts(1)
m_qcut.value_counts(1)
r_qcut.value_counts(1)

In [ ]:
# 열을 한번에 추가하는 법
rfm = rfm.assign(R=r_qcut,F=f_qcut,M=m_qcut)
rfm.head()

In [ ]:
rfm['RFM_segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)

In [ ]:
rfm['RFM_score'] = rfm[['R','F','M']].astype(int).sum(axis=1)

In [ ]:
plt.figure(figsize=(20,4))
plt.xticks(rotation=90)
sns.barplot(data=rfm.sort_values('RFM_segment'),x='RFM_segment',y='RFM_score',hue='RFM_segment')

In [ ]:
ax = plt.axes(projection='3d')
ax.scatter3D(rfm['R'],rfm['F'],rfm['M'])

In [ ]:
ax = plt.axes(projection='3d')
ax.scatter3D(rfm['Recency'],rfm['Frequency'],rfm['Monetary'])

In [ ]:
# qcut을 해줘서 'R','F','M' 값의 분포가 넓어짐
rfm.astype(int).hist(bins=5,figsize=(10,6));

In [ ]:
rfm.groupby(['RFM_score']).agg({'Recency':'mean',
                                'Frequency':'mean',
                                'Monetary':['mean','sum']}).style.background_gradient().format('{:,.0f}')

In [ ]:
# 덜 세분화 해보자 'silver','gold','platinum' 으로 고객군을 나눠보자
rfm['RFM_class'] = pd.qcut(x=rfm['RFM_score'],q=3,labels=['silver','gold','platinum'] )

In [ ]:
rfm.groupby('RFM_class')['RFM_score'].describe()

In [ ]:
sns.barplot(data=rfm,x='RFM_class',y='RFM_score',hue='RFM_class')

In [ ]:
sns.boxplot(data=rfm,x='RFM_class',y='RFM_score')

In [ ]:
# pointplot
sns.pointplot(data=rfm,x='R',y='RFM_score',hue='RFM_class')
sns.pointplot(data=rfm,x='F',y='RFM_score',hue='RFM_class')
sns.pointplot(data=rfm,x='M',y='RFM_score',hue='RFM_class')

In [ ]:
# RFM_class 별로 그룹화
# 'Recency','Frequency'의 평균을 구한다
# 'Montary'의 'mean','sum','count'를 구한
rfm.groupby('RFM_class').agg({'Recency':'mean',
                              'Frequency':'mean',
                              'Monetary':['mean','sum','count']}).style.background_gradient().format('{:,.0f}')

In [ ]:
# pairplot을 통해 rfm class 별로 분포를 시각화 해보자
sns.pairplot(data=rfm.sample(300),hue='RFM_class')

In [ ]:
corr = rfm.select_dtypes(include='number').corr()

In [ ]:
mask = np.triu(np.ones_like(corr))

In [ ]:
sns.heatmap(corr,annot=True,cmap='coolwarm',mask=mask)
# Recency는 최근일수록 점수를 높게 주었으니 음의 상관관계가 높게 나왔다.

In [ ]:
rfm.to_csv('rfm.csv')

In [ ]:
pd.read_csv('rfm.csv')